# Using the Amazon Braket tensor network simulator TN1

In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

<div class="alert alert-block alert-warning">
<b>Caution:</b> Running the code in this notebook will result in <a href="https://aws.amazon.com/braket/pricing/">usage fees</a> charged to your AWS account. The expected cost is less than $1.00 USD. Only run the notebook if you are comfortable with the costs. We recommend monitoring the Billing & Cost Management Dashboard on the AWS console and being aware that tasks involving a large number of qubits can be costly.
</div>

This tutorial serves as an in-depth introduction to TN1, the Amazon Braket tensor network simulator. TN1 was previously introduced in [Running quantum circuits on simulators notebook](../getting_started/1_Running_quantum_circuits_on_simulators/1_Running_quantum_circuits_on_simulators.ipynb). This tutorial explains what makes TN1 different from SV1, Braket's state vector simulator, and it discusses which use cases are well suited for tensor network simulations. We examine what circuit properties affect TN1's performance and how TN1 can be used to simulate some types circuits for many more qubits than SV1 can handle.

## How TN1 Works

A tensor network simulator models the execution of circuits on a quantum computer by representing each gate in the circuit as a *tensor*. Tensors generalize the concept of vectors and matrices to higher dimensions. The gates in the network form a graph. The simulator works by finding an efficient way to multiply all the different tensors on the graph during the _rehearsal_ stage and then, after a suitable multiplication sequence or _path_ is found, it performs these multiplications in the _contraction_ stage. For more information about TN1 and how it works, see the [TN1 docs](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-tn1).

## How is this different from SV1, the state vector simulator?

SV1 works differently -- SV1 simulates all the evolution of all amplitudes as gates are applied. This means that SV1 cannot simulate large numbers of qubits for any circuit, because the memory required becomes infeasible. However, this restriction does not necessarily apply to TN1. Because TN1 works by contracting gates, it is able to work only with worldlines through the circuit which are relevant to the final outcome. However, TN1 can be slower than SV1 for circuits with complex geometry. In circuits which include multi-qubit gates, gates with long range, or circuits with few qubits (fewer than 28), SV1 is often the better choice. To use TN1 effectively it is important to understand that circuit geometry can pose more of a barrier than simple qubit number, as we will see below.

## Can my circuit be simulated on TN1?

While SV1 will simulate every circuit within the service limits (i.e., smaller or equal than 34 qubits), TN1 can only decide if a circuit can be contracted after it has the full information from the rehearsal stage. In the best case, this enables you to simulate circuits of much larger size than SV1 (up to 50 qubits) but, on the flip side, that means that in some cases you might find that TN1 will terminate the simulation after the rehearsal if it finds that the projected contraction time exceeds its runtime limit. In this case the `failureReason` for the quantum task will be `Predicted runtime based on best contraction path found exceeds TN1 limit.` The rehearsal stage of TN1 is limited to 10 minutes, but in most cases you will find that TN1 will arrive at a decision much faster. 
    
As we will see below, if this situation occurs for a quantum task for which you have requested a large number of shots, the quantum task may be successful if you lower the shot count. It can also occur, albeit rarely, that the time to find a single contraction path candidate exceeds TN1's internal rehearsal runtime limit -- circuits for which this occurs are extremely unlikely to be contractable in reasonable time. In this case, the `failureReason` for the quantum task will be `No viable contraction path found.`
    
To learn more about why these two stages are present and what the simulator does in each, you can read the TN1 documentation [here](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html#braket-simulator-tn1).

<div class="alert alert-block alert-info">
<b>Note:</b> In the worst case, the TN1 runtime can scale linearly with the number of shots requested. It is strongly recommended to test your circuit or circuit class with a small number of shots first.
</div>

In [2]:
# general imports
import numpy as np
import math

import boto3
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, circuit, Gate, Instruction
from braket.aws import AwsDevice

tn_device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/tn1')
sv_device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')

## Two simple examples: The GHZ state and Quantum Fourier Transform

We already presented the GHZ example circuit in the [Running quantum circuits on simulators notebook](../getting_started/1_Running_quantum_circuits_on_simulators/1_Running_quantum_circuits_on_simulators.ipynb). Here, we'll compare the performance of SV1 and TN1 for this relatively simple circuit. The GHZ state is simple to prepare:

In [3]:
def ghz_circuit(n_qubits: int) -> Circuit:
    """
    Function to return simple GHZ circuit ansatz. Assumes all qubits in range(0, n_qubits-1)
    are entangled.

    :param int n_qubits: number of qubits
    :return: Constructed GHZ circuit
    :rtype: Circuit
    """

    circuit = Circuit()                          # instantiate circuit object
    circuit.h(0)                                 # add Hadamard gate on first qubit

    for ii in range(0, n_qubits-1):
        circuit.cnot(control=ii, target=ii+1)    # apply series of CNOT gates
    return circuit

We will simulate the measurement counts for this circuit on SV1 and TN1. SV1 can only simulate up to 34 qubits, but TN1 can work with substantially more in this case because of the circuit's geometry. In this case we will not run up to 34 qubits on SV1, because the runtime on that simulator can become quite long. 30 qubits is enough to see that TN1 can equal or outperform SV1 for circuits like GHZ, which has a simple, compact nearest-neighbor circuit geometry. Because the GHZ state is a "cat state", with only two possible measurement outcomes (all up or all down), it is easy for TN1 to explore all possible output bitstrings.

In [4]:
qubit_range = range(20, 31, 5)
tn_qubit_range = range(35, 51, 5)
n_shots     = 50
ghz_circs   = {}
sv_tasks    = {}
tn_tasks    = {}
sv_results  = {}
tn_results  = {}
for num_qubits in qubit_range:
    ghz = ghz_circuit(num_qubits)
    sv_tasks[num_qubits] = sv_device.run(ghz, shots=n_shots)
    tn_tasks[num_qubits] = tn_device.run(ghz, shots=n_shots)
    ghz_circs[num_qubits] = ghz

# Run qubit numbers which only TN1 supports
for num_qubits in tn_qubit_range:
    ghz = ghz_circuit(num_qubits)
    tn_tasks[num_qubits] = tn_device.run(ghz, shots=n_shots)
    ghz_circs[num_qubits] = ghz

for num_qubits in qubit_range:
    tn_status = tn_tasks[num_qubits].state()
    sv_status = sv_tasks[num_qubits].state()
    while tn_status != 'COMPLETED':
        tn_status = tn_tasks[num_qubits].state()
    while sv_status != 'COMPLETED':
        sv_status = sv_tasks[num_qubits].state()

    tn_results[num_qubits] = tn_tasks[num_qubits].result()
    sv_results[num_qubits] = sv_tasks[num_qubits].result()

    # get the running time of the quantum tasks
    sv_runtime = sv_results[num_qubits].additional_metadata.simulatorMetadata.executionDuration
    tn_runtime = tn_results[num_qubits].additional_metadata.simulatorMetadata.executionDuration

    # get the 'shots' parameter from metadata
    tn_num_shots = tn_results[num_qubits].task_metadata.shots
    sv_num_shots = sv_results[num_qubits].task_metadata.shots

    # get the measurement counts
    tn_counts = tn_results[num_qubits].measurement_counts
    sv_counts = sv_results[num_qubits].measurement_counts
    
    print("GHZ circuit:")
    print(ghz_circs[num_qubits])
    print('{}-qubit TN1 task {}.'.format(num_qubits,tn_status))
    print('Tensor network simulator:')
    print('This quantum task ran {} shots and the total runtime was {} ms'.format(tn_num_shots,tn_runtime))
    print("Measurement results: {}\n".format(tn_counts))
    print('{}-qubit SV1 task {}.'.format(num_qubits,sv_status))
    print('State vector simulator:')
    print('This quantum task ran {} shots and the total runtime was {} ms'.format(sv_num_shots,sv_runtime))
    print("Measurement results: {}\n".format(sv_counts))

for num_qubits in tn_qubit_range:
    tn_status = tn_tasks[num_qubits].state()
    while tn_status != 'COMPLETED':
        tn_status = tn_tasks[num_qubits].state()

    tn_results[num_qubits] = tn_tasks[num_qubits].result()

    # get the running time of the quantum tasks
    tn_runtime = tn_results[num_qubits].additional_metadata.simulatorMetadata.executionDuration

    # get the 'shots' parameter from metadata
    tn_num_shots = tn_results[num_qubits].task_metadata.shots

    # get the measurement counts
    tn_counts = tn_results[num_qubits].measurement_counts
    
    # we will not print the circuits here as they are quite large
    print('{}-qubit TN1 task {}.'.format(num_qubits,tn_status))
    print('Tensor network simulator:')
    print('This quantum task ran {} shots and the total runtime was {} ms'.format(tn_num_shots,tn_runtime))
    print("Measurement results: {}\n".format(tn_counts))

GHZ circuit:
T   : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|
                                                         
q0  : -H-C-----------------------------------------------
         |                                               
q1  : ---X-C---------------------------------------------
           |                                             
q2  : -----X-C-------------------------------------------
             |                                           
q3  : -------X-C-----------------------------------------
               |                                         
q4  : ---------X-C---------------------------------------
                 |                                       
q5  : -----------X-C-------------------------------------
                   |                                     
q6  : -------------X-C-----------------------------------
                     |                                   
q7  : ---------------X-C---------------------------------
 

35-qubit TN1 task COMPLETED.
Tensor network simulator:
This quantum task ran 50 shots and the total runtime was 2284 ms
Measurement results: Counter({'11111111111111111111111111111111111': 29, '00000000000000000000000000000000000': 21})

40-qubit TN1 task COMPLETED.
Tensor network simulator:
This quantum task ran 50 shots and the total runtime was 2412 ms
Measurement results: Counter({'1111111111111111111111111111111111111111': 29, '0000000000000000000000000000000000000000': 21})

45-qubit TN1 task COMPLETED.
Tensor network simulator:
This quantum task ran 50 shots and the total runtime was 2377 ms
Measurement results: Counter({'000000000000000000000000000000000000000000000': 26, '111111111111111111111111111111111111111111111': 24})

50-qubit TN1 task COMPLETED.
Tensor network simulator:
This quantum task ran 50 shots and the total runtime was 2434 ms
Measurement results: Counter({'00000000000000000000000000000000000000000000000000': 25, '11111111111111111111111111111111111111111111111

There are many circuits which can be efficiently simulated by TN1 even up to very large qubit counts. Another example is the quantum Fourier transform (QFT) and its inverse, shown in [the QFT notebook](../advanced_circuits_algorithms/Quantum_Fourier_Transform/Quantum_Fourier_Transform.ipynb). TN1 is able to efficiently simulate the QFT on an input state of |00..00> because in this case it amounts to a simple rotation, even up to many qubits:

In [5]:
@circuit.subroutine(register=True)
def qft(qubits):    
    """
    Construct a circuit object corresponding to the Quantum Fourier Transform (QFT)
    algorithm, applied to the argument qubits.  Does not use recursion to generate the QFT.
    
    Args:
        qubits (int): The list of qubits on which to apply the QFT
    """
    qftcirc = Circuit()

    # get number of qubits
    num_qubits = len(qubits)
    
    for k in range(num_qubits):
        # First add a Hadamard gate
        qftcirc.h(qubits[k])
    
        # Then apply the controlled rotations, with weights (angles) defined by the distance to the control qubit.
        # Start on the qubit after qubit k, and iterate until the end.  When num_qubits==1, this loop does not run.
        for j in range(1,num_qubits - k):
            angle = 2*math.pi/(2**(j+1))
            qftcirc.cphaseshift(qubits[k+j],qubits[k], angle)
            
    # Then add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits/2)):
        qftcirc.swap(qubits[i], qubits[-i-1])
        
    return qftcirc

In [6]:
qubit_range = range(20, 41, 10)
tn_tasks    = {}
tn_results  = {}
for num_qubits in qubit_range:
    # generate QFT circuit
    qft_circ = qft(range(num_qubits))
    tn_tasks[num_qubits] = tn_device.run(qft_circ, shots=n_shots)

for num_qubits in qubit_range:
    tn_status = tn_tasks[num_qubits].state()
    while tn_status != 'COMPLETED':
        tn_status = tn_tasks[num_qubits].state()

    tn_results[num_qubits] = tn_tasks[num_qubits].result()
    # get the running time of the quantum task
    tn_runtime = tn_results[num_qubits].additional_metadata.simulatorMetadata.executionDuration

    # get the measurement counts
    tn_counts = tn_results[num_qubits].measurement_counts
    
    print('{}-qubit task {}.'.format(num_qubits,tn_status))
    print('QFT:')
    print('This quantum task ran {} shots and the total runtime was {} ms'.format(tn_num_shots,tn_runtime))
    print("Measurement results: {}\n".format(tn_counts))

20-qubit task COMPLETED.
QFT:
This quantum task ran 50 shots and the total runtime was 2210 ms
Measurement results: Counter({'10111011100101010111': 1, '00111001010110001110': 1, '01011010001110100100': 1, '01000110101110001101': 1, '00111111100101100100': 1, '10100011010111101001': 1, '00100000010010100001': 1, '10110011111011100101': 1, '01110011001001110001': 1, '01010011110010110110': 1, '10010000110100010100': 1, '10000101000110011000': 1, '01101011110100010111': 1, '10111111100011010011': 1, '10110000101110011110': 1, '10000111111001001000': 1, '10011010001001001011': 1, '01000010110110101111': 1, '00110011110110011000': 1, '11001101000001101000': 1, '11110110111001101011': 1, '00110010101111011011': 1, '11000100000001110101': 1, '01011011110001111010': 1, '00010101000010111010': 1, '11000001111110111100': 1, '10000110010111101100': 1, '11010110101110110010': 1, '01010111000011010110': 1, '01010100001110100101': 1, '00111100000101101111': 1, '00010101101100001010': 1, '1000111010

## Circuit geometry is extremely important for TN1

We'll now examine a type of circuit that is harder for the tensor network simulator: local Hayden-Preskill circuits. "Local" here means that multi-qubit gates only act on qubits which are nearest-neighbors. In this case, we will simulate a one-dimensional chain, and so any 2-qubit gates will act on qubits `q` and `q+1`. These circuits are generated in the following way:

For each of `N` gates:
- Choose a single qubit gate with 50% chance
    - Choose between x-, y-, or z-rotations and Hadamard with equal likelihood.
    - If a rotation is chosen, the angle is chosen from a uniform random distribution between `0` and `2*pi`.
    - The qubit `q` to which to apply the gate is chosen randomly with equal likelihood from among all the available `N` qubits
- Choose a CZ gate with 50% chance
    - The qubit `q` to which to apply the control is chosen randomly with equal likelihood from among `N-1` qubits. The `Z` gate is applied to `q+1`.

Circuits of this type are effective at generating entanglement among qubits. We can see this by scaling the number of gates applied quadratically with the number of qubits. This is enough gates to spread entanglement throughout all the qubits. We see that the circuit becomes more time consuming to simulate as we apply more gates.

In [7]:
@circuit.subroutine(register=True)
def local_Hayden_Preskill(num_qubits, num_gates):
    hp_circ = Circuit()
    """Yields the circuit elements for a scrambling unitary.
    Generates a circuit with numgates gates by laying down a
    random gate at each time step.  Gates are chosen from single
    qubit unitary rotations by a random angle, Hadamard, or a 
    controlled-Z between a qubit and its nearest neighbor (i.e.,
    incremented by 1)."""
    qubits = range(num_qubits)
    for i in range(num_gates):
        if np.random.random_sample() > 0.5:
            """CZ between a random qubit and the next qubit to its left."""
            a = np.random.choice(range(len(qubits)-1), 1, replace=True)[0]
            hp_circ.cz(qubits[a],qubits[a+1])
        else:
            """Random single qubit rotation."""
            angle = np.random.uniform(0, 2 * math.pi)
            qubit = np.random.choice(qubits,1,replace=True)[0]
            gate  = np.random.choice([Gate.Rx(angle), Gate.Ry(angle), Gate.Rz(angle), Gate.H()], 1, replace=True)[0]
            hp_circ.add_instruction(Instruction(gate, qubit))
    return hp_circ

## Visualizing the Hayden-Preskill circuit

Let's examine the geometry of some HP circuits of varying depths. We can see that the deeper the circuit, the more likely it is for (mediated) connections to exist among all qubits.

In [8]:
num_qubits = 5
for num_gates in range(5, 26, 5):
    print('')
    print(f"HAYDEN PRESKILL CIRCUIT WITH {num_gates} GATES:")
    my_hp_circ = local_Hayden_Preskill(num_qubits, num_gates)
    print(my_hp_circ)


HAYDEN PRESKILL CIRCUIT WITH 5 GATES:
T  : |   0    |   1    |2|
                          
q0 : -C-------------------
      |                   
q1 : -Z-------------------
                          
q2 : -Ry(6.24)------------
                          
q3 : -C--------Rx(2.63)-C-
      |                 | 
q4 : -Z-----------------Z-

T  : |   0    |   1    |2|

HAYDEN PRESKILL CIRCUIT WITH 10 GATES:
T  : |0|   1    |2|3|   4    |   5    |
                                       
q1 : --------------C-Rx(4.33)-Rx(0.81)-
                   |                   
q2 : -H----------C-Z-C-----------------
                 |   |                 
q3 : -C-H--------Z-C-Z-----------------
      |            |                   
q4 : -Z-Rx(1.79)---Z-------------------

T  : |0|   1    |2|3|   4    |   5    |

HAYDEN PRESKILL CIRCUIT WITH 15 GATES:
T  : |   0    |1|   2    |3|   4    |5|6|
                                         
q0 : ---------------------C-Rx(2.12)-----
                          |  

## Experimenting with Hayden-Preskill circuits

We will examine runtimes for various depths of circuits at relatively low (for TN1) qubit counts. This is to ensure that our job finishes in a reasonable amount of time. We'll examine the measurement counts at the end of each simulation. Because these are random circuits, we should not expect to see the measurement counts highly concentrated in a few outcomes.

In [9]:
num_qubits  = 20
n_shots     = 5
gate_range  = [250, 500]
tn_tasks    = {}
tn_results  = {}
for gates in gate_range:
    # construct HP circuit
    circ = Circuit()
    # ensure the HP circuit is runnable -- circuits must have depth <= 100
    while True:
        circ = local_Hayden_Preskill(num_qubits, gates)
        if circ.depth <= 100:
            break
    tn_tasks[circ.depth] = tn_device.run(circ, shots=n_shots)

for depth in tn_tasks.keys():
    tn_status = tn_tasks[depth].state()
    while tn_status != 'COMPLETED':
        tn_status = tn_tasks[depth].state()

    tn_results[depth] = tn_tasks[depth].result()
    # get the running time of the task
    tn_runtime = tn_results[depth].additional_metadata.simulatorMetadata.executionDuration

    # get the measurement counts
    tn_counts = tn_results[depth].measurement_counts
    
    print('{}-qubit {}-depth task {}.'.format(num_qubits,depth,tn_status))
    print('Hayden-Preskill circuit:')
    print('This task ran {} shots and the total runtime was {} ms'.format(n_shots,tn_runtime))
    print("Measurement results: {}\n".format(tn_counts))

20-qubit 48-depth task COMPLETED.
Hayden-Preskill circuit:
This task ran 5 shots and the total runtime was 6795 ms
Measurement results: Counter({'11010111111101110111': 1, '11000011111000101101': 1, '11100110100100010110': 1, '11000010001100101111': 1, '10000001011010111100': 1})

20-qubit 69-depth task COMPLETED.
Hayden-Preskill circuit:
This task ran 5 shots and the total runtime was 22773 ms
Measurement results: Counter({'00110010000101010000': 1, '00010001010101000001': 1, '11011100010101000111': 1, '11101100011011001100': 1, '10001110000001110111': 1})



## The effect of shot counts on runtimes in TN1

In order to generate samples from a quantum circuit, TN1 first partitions the qubits into groups, and then contracts each group in turn, generating a prediction for the current group's configuration based on the results of previously encountered groups. Because of this, in the worst case, the time to generate `n` shots may scale linearly with `n`. However, if the number of possible outcomes is small (as it is in the GHZ or QFT case), the runtime is effectively constant no matter the number of shots. We will examine this behavior below. It's therefore important to understand that your job may be rejected if the time to contract all the shots you have requested is too large. In this case your task will finish with a `FAILED` message, and the `failureReason` will be `Predicted runtime based on best contraction path found exceeds TN1 limit.` -- in this case you can attempt to retry the computation with fewer shots.

<div class="alert alert-block alert-info">
<b>Note:</b> Because the runtime of the quantum task can scale linearly with the number of shots in the worst case, it is <b>strongly advised</b> that users run their circuits with a small number of shots (20 or fewer) to explore typical runtimes for their circuit <b>before</b> running the circuit for many shots.
</div>

In [10]:
num_qubits = 25
ghz_circ = ghz_circuit(num_qubits)
qft_circ = qft(range(num_qubits))
hp_circ  = Circuit()
for _i in range(10):
    circ = local_Hayden_Preskill(num_qubits, 500)
    if circ.depth <= 100:
        hp_circ = circ
        break
ghz_tasks    = {}
ghz_results  = {}
qft_tasks    = {}
qft_results  = {}
hp_tasks     = {}
hp_results   = {}
for n_shots in [5, 20]:
    ghz_tasks[n_shots] = tn_device.run(ghz_circ, shots=n_shots)
    qft_tasks[n_shots] = tn_device.run(qft_circ, shots=n_shots)
    hp_tasks[n_shots]  = tn_device.run(hp_circ, shots=n_shots)

for n_shots in [5, 20]:
    ghz_status = ghz_tasks[n_shots].state()
    while ghz_status != 'COMPLETED':
        ghz_status = ghz_tasks[n_shots].state()

    qft_status = qft_tasks[n_shots].state()
    while qft_status != 'COMPLETED':
        qft_status = qft_tasks[n_shots].state()

    hp_status  = hp_tasks[n_shots].state()
    while hp_status != 'COMPLETED':
        hp_status = hp_tasks[n_shots].state()

    ghz_results[n_shots] = ghz_tasks[n_shots].result()
    qft_results[n_shots] = qft_tasks[n_shots].result()
    hp_results[n_shots]  = hp_tasks[n_shots].result()
    # get the running time of the task
    ghz_runtime = ghz_results[n_shots].additional_metadata.simulatorMetadata.executionDuration
    qft_runtime = qft_results[n_shots].additional_metadata.simulatorMetadata.executionDuration
    hp_runtime  = hp_results[n_shots].additional_metadata.simulatorMetadata.executionDuration

    print('GHZ task ran {} shots and the total runtime was {} ms'.format(n_shots,ghz_runtime))
    print('QFT task ran {} shots and the total runtime was {} ms'.format(n_shots,qft_runtime))
    print('HP task ran {} shots and the total runtime was {} ms'.format(n_shots,hp_runtime))

GHZ task ran 5 shots and the total runtime was 1696 ms
QFT task ran 5 shots and the total runtime was 2940 ms
HP task ran 5 shots and the total runtime was 40343 ms
GHZ task ran 20 shots and the total runtime was 1315 ms
QFT task ran 20 shots and the total runtime was 2916 ms
HP task ran 20 shots and the total runtime was 38092 ms


In [11]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD")

Quantum Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 150, 'tasks': {'COMPLETED': 3}, 'execution_duration': datetime.timedelta(seconds=15, microseconds=990000), 'billed_execution_duration': datetime.timedelta(seconds=21, microseconds=494000)}, 'arn:aws:braket:::device/quantum-simulator/amazon/tn1': {'shots': 585, 'tasks': {'COMPLETED': 18}, 'execution_duration': datetime.timedelta(seconds=190, microseconds=695000), 'billed_execution_duration': datetime.timedelta(seconds=197, microseconds=787000)}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 0.933 USD
